In [1]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import folium
from folium.plugins import HeatMap
from shapely.geometry import MultiPoint

In [3]:
import os
import sys

from os.path import dirname
UTILS_PATH=os.environ.get('REPO') + "/notebooks/commons"
sys.path.append(dirname(UTILS_PATH))

from commons import check_args, gen_paths, coordinates_bounds

In [4]:
import math

def safe_pow(e):
    result = math.inf
    try:
        result = float(2**e)
        return result
    except OverflowError as error:
        return result
    

In [5]:
def load_posts(input_path):
    df = pd.read_parquet(input_path)
    return df

In [6]:
if __name__ == "__main__":
    city = "manizales"
    property_type = "apartamentos"
    post_type = "venta"
    
    #city, property_type, post_type = check_args()
    base_path = os.environ.get('REPO')
    
    raw_path, clean_path = gen_paths(city, property_type, post_type)
    
    raw_path = base_path + raw_path
    clean_path = base_path + clean_path

In [7]:
MSG = """
Hay {} publicaciones de {} en total
"""

posts = load_posts(clean_path)
print(MSG.format(posts.shape[0], post_type))
posts.head()


Hay 3004 publicaciones de venta en total



,surface,rooms,baths,garages,price,latitude,description,longitude,location,url,admon,estrato,antiguedad,fid,city,property_type,post_type,price_m2
1,54.0,2.0,2.0,1.0,220.0,5.059000,"Apartamento ubicado cerca a Homecenter, constr...",-75.478996,Manizales La Toscana,https://www.fincaraiz.com.co/apartamento-en-ve...,0.0,4.0,1 a 8,5478594,manizales,apartamentos,venta,4.074074
2,72.0,3.0,3.0,1.0,280.0,5.043152,"Se cede apartamento 606 de la Torre 1 , 3 habi...",-75.473587,Manizales Avda Alberto Mendoza Frente a Expofe...,https://www.fincaraiz.com.co/apartamento-en-ve...,0.0,5.0,desconocido,5720132,manizales,apartamentos,venta,3.888889
3,160.0,4.0,3.0,1.0,290.0,5.050346,"Apartamento duplex, remodelado en La Camelia, ...",-75.484329,Manizales Palermo,https://www.fincaraiz.com.co/apartamento-en-ve...,300000.0,6.0,16 a 30,2870506,manizales,apartamentos,venta,1.812500
4,93.0,3.0,2.0,1.0,420.0,5.061000,"Bello y cómodo apartamento, ubicado sobre la A...",-75.490990,Manizales Av Santander,https://www.fincaraiz.com.co/apartamento-en-ve...,240000.0,6.0,1 a 8,5288572,manizales,apartamentos,venta,4.516129
6,65.0,3.0,2.0,1.0,150.0,5.064650,Inmobiliaria Giraldo Castaño ofrece apartament...,-75.507553,Manizales palonegro,https://www.fincaraiz.com.co/apartamento-en-ve...,0.0,3.0,1 a 8,5149671,manizales,apartamentos,venta,2.307692


## Mapa de calor precios

In [8]:
posts["price"].describe()

count    3004.000000
mean      267.864463
std       173.196104
min        55.000000
25%       180.000000
50%       230.000000
75%       310.000000
max      3750.000000
Name: price, dtype: float64

In [9]:
folium_hmap = folium.Map(location=[
    np.mean([coordinates_bounds[city]["lat"]["lower"], coordinates_bounds[city]["lat"]["upper"]]),
    np.mean([coordinates_bounds[city]["lon"]["lower"], coordinates_bounds[city]["lon"]["upper"]])
], zoom_start=13, tiles="OpenStreetMap")

posts.loc[:, "price_exploded"] =  posts["price"].apply(safe_pow)

max_amount = posts['price_exploded'].max()

hm_wide = HeatMap( 
    posts[["latitude", "longitude", "price_exploded"]],
    min_opacity=0.2,
    max_val=max_amount,
    radius=7, blur=1, 
    max_zoom=15, 
    gradient={.1:'yellow', .5: 'orange',  1: 'red'}
)

folium_hmap.add_child(hm_wide)

## Mapa de calor número de publicaciones

In [10]:
from sklearn.cluster import DBSCAN

In [11]:
coords = posts[['latitude','longitude']].values

kms_per_radian = 6371.0088
epsilon = 0.1 / kms_per_radian

min_samples = 20

In [12]:
## Geo clustering with examples
# https://stackoverflow.com/questions/24762435/clustering-geo-location-coordinates-lat-long-pairs-using-kmeans-algorithm-with
# https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/

db = DBSCAN(eps=epsilon, min_samples=min_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))

In [13]:
centroids = []
clusters_size = len(db.labels_)

for n in range(clusters_size):
    cluster = posts[['latitude','longitude']][db.labels_ == n].copy()
    if len(cluster) > 0:
        cluster.loc[:, "cluster_id"] = n
        
        multi_point = MultiPoint(cluster.values)
        cluster.loc[:, "cluster_latitude"] = multi_point.centroid.x
        cluster.loc[:, "cluster_longitude"] = multi_point.centroid.y
        
    centroids.append(cluster)
        
clusters_df = pd.concat(centroids)
clusters_df = clusters_df.reset_index(drop=True)
clusters_df.head()

,latitude,longitude,cluster_id,cluster_latitude,cluster_longitude
0,5.050346,-75.484329,0.0,5.050966,-75.484239
1,5.050276,-75.483200,0.0,5.050966,-75.484239
2,5.049675,-75.483017,0.0,5.050966,-75.484239
3,5.052295,-75.483383,0.0,5.050966,-75.484239
4,5.048810,-75.482910,0.0,5.050966,-75.484239


In [14]:
clusters_coordinates = clusters_df[["cluster_id", "latitude", "longitude"]].groupby(["latitude", "longitude"]).count()

coordinates = list(zip(*clusters_coordinates.index))
clusters_coordinates.loc[:, "latitude"] = coordinates[0]
clusters_coordinates.loc[:, "longitude"] = coordinates[1]

clusters_coordinates.loc[:, "count"] = clusters_coordinates["cluster_id"].apply(safe_pow)
clusters_coordinates = clusters_coordinates.reset_index(drop=True)

clusters_coordinates.head()

,cluster_id,latitude,longitude,count
0,1,5.040416,-75.477585,2.0
1,1,5.040818,-75.478706,2.0
2,2,5.041254,-75.478043,4.0
3,1,5.041288,-75.477776,2.0
4,1,5.041369,-75.478249,2.0


In [15]:
folium_hmap = folium.Map(location=[
    np.mean([coordinates_bounds[city]["lat"]["lower"], coordinates_bounds[city]["lat"]["upper"]]),
    np.mean([coordinates_bounds[city]["lon"]["lower"], coordinates_bounds[city]["lon"]["upper"]])
], zoom_start=13, tiles="OpenStreetMap")

max_amount = clusters_coordinates['count'].max()

hm_wide = HeatMap( 
    clusters_coordinates[["latitude", "longitude", "count"]],
    min_opacity=0.2,
    max_val=max_amount,
    radius=7, blur=1, 
    max_zoom=15, 
    gradient={.1:'yellow', .5: 'orange',  1: 'red'}
)

folium_hmap.add_child(hm_wide)